In [8]:
import numpy as np
import os
import re
from lxml import etree
import xml.etree.ElementTree as ET

In [2]:
# Prediciting code
# yaltai kraken --device cpu -I "*.jpg" --suffix ".xml" segment --yolo /Users/Theo/Desktop/seg_models/big_seg_model_v2/weights/best.pt -i blla.mlmodel
# kraken -a -I '*.xml' -o _ocr.xml -f xml ocr -m kraken_htrtrained.mlmodel


2024-01-22 11:20:53.507817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
TensorFlow version 2.11.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.10.0 is the most recent version that has been tested.
Loading ANN /usr/local/anaconda3/lib/python3.9/site-packages/kraken/blla.mlmodel	Segmenting	Using cache found in /Users/Theo/.cache/torch/hub/ultralytics_yolov5_v6.2
YOLOv5 🚀 2023-3-3 Python-3.9.13 torch-1.13.1 CPU

Model summary (fused): 218 layers, 25847287 parameters, 0 gradients, 78.7 GFLOPs
Adding AutoShape... 
✗
[01/22/24 11:21:44] ERROR    Failed processing ED.jpg: 1    ]8;id=655473;file:///usr/local/anaconda3/lib/python3.9/site-packages/yaltai/kraken_yaltai.

In [13]:
directory = "/Users/Theo/dev/matricule_htr/"

In [46]:
#Code that displays the file paths in the alto documents
for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        # Load the XML file into an ElementTree object
        tree = ET.parse(os.path.join(directory, filename))
        
        # Get the root element
        root = tree.getroot()
        
        # Find the fileName element and print its text content
        file_name_elems = root.findall(".//{http://www.loc.gov/standards/alto/ns-v4#}fileName")
        for file_name_elem in file_name_elems:
            print(file_name_elem.text)

ED.jpg
pinero2.jpg
pinero.jpg


In [47]:
#Code that changes the file paths in the alto documents
ns_dict = {
    'alto': 'http://www.loc.gov/standards/alto/ns-v4#',
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
}
ns_prefix = '{http://www.loc.gov/standards/alto/ns-v4#}'

# Loop over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        # Load the XML file into an ElementTree object
        tree = ET.parse(os.path.join(directory, filename))
        
        # Find the root element and remove the namespace prefix
        root = tree.getroot()
        if root.tag.startswith(ns_prefix):
            root.tag = 'alto'
            root.attrib = {
                'xmlns:xsi': ns_dict['xsi'],
                'xmlns': ns_dict['alto'],
                'xsi:schemaLocation': 'http://www.loc.gov/standards/alto/ns-v4# http://www.loc.gov/standards/alto/v4/alto-4-3.xsd'
            }
        
        # Find all elements with the ALTO namespace
        for elem in tree.findall('.//{http://www.loc.gov/standards/alto/ns-v4#}*'):
            # Remove the namespace prefix from the element tag
            elem.tag = re.sub(r'{.*}', '', elem.tag)
        
        # Find all fileName elements and modify their text content
        for file_name_elem in tree.findall('.//fileName', ns_dict):
            # Remove all characters and signs in between any slashes in the filename, and return only the filename itself
            file_name_elem.text = re.sub(r'^.*/', '', file_name_elem.text)
            
        # Write the modified XML file to disk
        tree.write(os.path.join(directory, filename), encoding='UTF-8', xml_declaration=True)
        

In [14]:
from functions import *

In [83]:
directory = "/Users/Theo/dev/matricule_htr/tests/"
output = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        # Load the XML file into an ElementTree object
        tree = ET.parse(os.path.join(directory, filename))
        region_list = []
        region_value = []
        for region in tree.findall(".//{http://www.loc.gov/standards/alto/ns-v4#}OtherTag"):
            if region.attrib['LABEL'] != 'default':
                region_list.append(region.attrib['LABEL'])
                region_value.append(region.attrib['ID'])
        
        regions_dict = {region_list[i]: region_value[i] for i in range(len(region_list))}
        
        result = {}
        for key, value in regions_dict.items():
            line1 = ".//{http://www.loc.gov/standards/alto/ns-v4#}"
            line2 = f"TextBlock[@TAGREFS='{value}']/"
            line3 = "{http://www.loc.gov/standards/alto/ns-v4#}TextLine/"
            lines = tree.findall(line1 + line2 + line3 + "{http://www.loc.gov/standards/alto/ns-v4#}String")
                
            cat = []
        
            for content in lines:
                cat.append(content.attrib['CONTENT'])
            
            result[f'{key}'] = cat
            df_dictionary = pd.DataFrame([result])
            
        output = pd.concat([output, df_dictionary], ignore_index=True)


In [84]:
output

,nom,etat_civil,decision,details,cassier,campagnes,num_mat,signalement,affectation,adresse
0,"[ , Aarice, Henri, Maccel, ce]","[29, novembre, 1900, Alécachon, u, dil, Drcach...","[18, AOrcachon, 31, 1^ere]","[Renvoyé, en, disponibilité, le, 2, Mars, 19...","[ , le, 1vvil, 1922oet, mtl, Mhaseres, , a, f...","[Marse, en, Guerre, du, 11-4-1220, au, 23.11.1...","[CISt, 1220]",[60],"[rég^t, d'aviation, du, maro, 37^e, regt, d'o...",NaN
1,"[Cranse, Marie, François, Edouard]","[1, Novemcbe, 1898, e, Bouscal, e, la, Seronde...","[roir, NE, RECRETELEAT, DE, BORBEAU, EI, ADRES...","[Engagé́, volontaire, pour, la, duree, de, la,...",NaN,NaN,"[9, 19, de]",NaN,"[58^e, R, MC., 7e4d, 22, d°, 34644, 49^e, R.M....","[21, 12-.21, 25, Rue, desuhonse, B°, R, 110-24..."
2,"[7, ?, X, Royer, pours, Drgiole]","[u, dit, 3e, cAoul, 189/., Covrbes, Gabrerl, e...","[31, 8, Gatarrel]","[Incorporé, au, 14e, rég^t, de, Art^e, le, 16...",NaN,"[arméés, du, 2., C.18, du, 6-7-18, arirées, du...","[^on, de, chnoin, 68, 2311]","[chétains, Eusgice, 11e, 9ef, a, 1ex, 21]","[eapecte, 01, aO^tis, le, 20-10, (54, ceuve]","[5128, 10, ece, de, S^on, d'armeare, Mort, C]"
